In [1]:
#导入数据
from sklearn.datasets import fetch_20newsgroups
news=fetch_20newsgroups(subset='all')
X,y=news.data, news.target


In [2]:
from bs4 import BeautifulSoup
import nltk, re

In [3]:
#将每条新闻的句子剥离出来,并且返回一个句子
def news_to_sentences(news):
    news_text=BeautifulSoup(news).get_text()
    tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences=tokenizer.tokenize(news_text)
    sentences=[]
    for sent in raw_sentences:
        sentences.append(re.sub('[^a-zA-z]',' ',sent.lower().strip()).split())
    return sentences



In [4]:
sentences=[]

#剥离句子，训练
for x in X:
    sentences+=news_to_sentences(x)

In [5]:
from gensim.models import word2vec
#词向量的纬度
num_features=300
#词汇的额度
min_word_count=20
#并行化数量
num_workers=2
#词向量训练时，上下文窗口大小
context=5
downsampling=1e-3

E:\anaconda\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
#训练词向量模型
model=word2vec.Word2Vec(sentences,workers=num_workers,vector_size=num_features,min_count=min_word_count,window=context,sample=downsampling)
model.init_sims(replace=True) #加快模型训练速度，

<ipython-input-7-74ea232eb94f>:3: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True) #加快模型训练速度，


In [10]:
#利用训练好的模型，看文本中morning最相似的10个词汇
model.wv.most_similar('morning')

[('afternoon', 0.8198807239532471),
 ('evening', 0.7801409959793091),
 ('weekend', 0.7744500041007996),
 ('saturday', 0.7242053747177124),
 ('friday', 0.7057297825813293),
 ('sunday', 0.696782112121582),
 ('night', 0.6858790516853333),
 ('summer', 0.6618945598602295),
 ('week', 0.6534996628761292),
 ('thursday', 0.6434074640274048)]

In [11]:
model.wv.most_similar('email')

[('mail', 0.7134678363800049),
 ('replies', 0.6776095032691956),
 ('address', 0.6522341966629028),
 ('contact', 0.6515761017799377),
 ('request', 0.6429516077041626),
 ('send', 0.6296660900115967),
 ('subscription', 0.6185706853866577),
 ('listserv', 0.6096134781837463),
 ('archie', 0.605798602104187),
 ('compuserve', 0.604825496673584)]